# Problem 1

### A) Prove that E step update on membership (\pi) achieves the minimum objective given the current centroids( \mu)

By updating the membership, we can move some datapoints to other appropriate cluster. By doing this, we can position each datapoint to the nearest centroid. If E step doesn't update on membership achieves the minimum objective, than the objective would remain the same or increase and wouldn't be placed near the closest centroid.

#### Add) Given a datapoints X, argmin function minimizes the within-cluster sum of square of difference between each datapoint and the centroid.



### B) Prove that M step update on centroids (\mu) achievess the minimum objective given the current memberships( \pi)

In the M-Step, mean minimizes total distance given the current clustering: M-step uses argmin function that returns the minimum value from the given axis. By going through EM loop as a whole, the objective always decreases. To make an optimal solution it is necessary to minimize the summed distance between every point and its centroid.

#### Add) Take partial derivative of μk and set to zero

### C) Explain why KMeans has to stop (converge), but not necessarily to the global minimum objective value.

The objective function optimized by the K-Means is not convex. It suffers from many local optima and it is sensitive to the initial mu and pi values. When we're given the bad seed in the first place, there's a high possibility it will result in bad clustering result with poor convergence.


# Problem 2

In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt

from sklearn import datasets
from keras.datasets import mnist
from collections import defaultdict, Counter
from sklearn.preprocessing import normalize
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.feature_extraction.text import TfidfVectorizer

## Section A - MNIST

In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = normalize(X_train.reshape(60000, 784))
X_test = normalize(X_test.reshape(10000, 784))
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


In [3]:
def display_digit(digit, title = ""):
    """
    graphically displays a 784x1 vector, representing a digit
    """
    plt.figure()
    fig = plt.imshow(digit)
    fig.set_cmap('gray_r')
    fig.axes.get_xaxis().set_visible(False)
    fig.axes.get_yaxis().set_visible(False)
    if title != "":
        plt.title("Inferred label: " + str(title))

#### K-Means Tools

In [4]:
def initial_centroids(X_train, k, centroids_map={}):
    # initialize centroids with random value
    for k_val, train_data in enumerate(random.choices(X_train, k=k)):
        centroids_map[k_val] = train_data
    return centroids_map

def update_centroids(cluster_map, centroids_map={}):
    # tune centroids based on the updated cluster info
    for key, val in cluster_map.items():
        centroids_map[key] = sum(val) / len(val)
    return centroids_map

def k_means_clustering(X_train, y_train, k, max_iter):
    final_cluster_map, centroids_map = {}, {}
    old_cluster_distances_sum = float('inf')
    centroids_map = initial_centroids(X_train, k)
    for i in range(max_iter):
        centroids_val = list(centroids_map.values())
        # compute pairwise minimum distance
        cluster_labels, cluster_distances = pairwise_distances_argmin_min(X_train, centroids_val)

        # set stop rule
        if old_cluster_distances_sum - sum(cluster_distances) <= 0.001:
            print('converged at iter', i)
            break
        old_cluster_distances_sum = sum(cluster_distances)

        cluster_map = defaultdict(list)
        label_cluster_map = defaultdict(list)
        
        # key: cluster label / val: mnist label value within the cluster
        for cluster_label, train_label in zip(cluster_labels, y_train):
            label_cluster_map[cluster_label].append(train_label)

        for x_train_val, cluster_label in zip(X_train, cluster_labels):
            cluster_map[cluster_label].append(x_train_val)

        # tune centroids given the updated cluster map
        centroids_map = update_centroids(cluster_map)
    return label_cluster_map, cluster_distances

def evaluate_purity_score(cluster_distances, label_cluster_map):
    sum_count = 0
    for label, value in label_cluster_map.items():
        most_frequent_label_count = Counter(value).most_common(1)[0][1]
        sum_count += most_frequent_label_count
    purity_score = sum_count / len(X_train) * 100
    print("evaluated objective is {}".format(sum(cluster_distances)))
    print("evaluated purity score is {}%".format(purity_score))

def evaluate_gini_index(X_train, label_cluster_map):
    sum_val = 0
    for label, val in label_cluster_map.items():
        sum_val += (Counter(val).most_common(1)[0][1] / len(X_train)) ** 2
    return 1 - sum_val

In [5]:
label_cluster_map, cluster_distances = k_means_clustering(X_train, y_train, k=5, max_iter=100)
evaluate_purity_score(cluster_distances, label_cluster_map)
evaluate_gini_index(X_train, label_cluster_map)

converged at iter 56
evaluated objective is 41971.14899429267
evaluated purity score is 39.82333333333333%


0.9662455666666667

In [6]:
label_cluster_map, cluster_distances = k_means_clustering(X_train, y_train, k=10, max_iter=100)
evaluate_purity_score(cluster_distances, label_cluster_map)
evaluate_gini_index(X_train, label_cluster_map)

converged at iter 57
evaluated objective is 39360.4540000393
evaluated purity score is 61.00833333333333%


0.9609515402777777

In [7]:
label_cluster_map, cluster_distances = k_means_clustering(X_train, y_train, k=20, max_iter=100)
evaluate_purity_score(cluster_distances, label_cluster_map)
evaluate_gini_index(X_train, label_cluster_map)

converged at iter 71
evaluated objective is 37199.58953827097
evaluated purity score is 72.97666666666667%


0.9717085377777778

## Section B - Fashion

In [5]:
from keras.datasets import fashion_mnist

(X_train, y_train) , (X_test, y_test)= fashion_mnist.load_data()

X_train = normalize(X_train.reshape(60000, 784))
X_test = normalize(X_test.reshape(10000, 784))
print(X_train.shape)
print(y_train.shape)

(60000, 784)
(60000,)


In [6]:
label_cluster_map, cluster_distances = k_means_clustering(X_train, y_train, k=5, max_iter=100)
evaluate_purity_score(cluster_distances, label_cluster_map)
evaluate_gini_index(X_train, label_cluster_map)

converged at iter 27
evaluated objective is 28453.509329888904
evaluated purity score is 45.81166666666667%


0.9579964941666667

In [7]:
label_cluster_map, cluster_distances = k_means_clustering(X_train, y_train, k=10, max_iter=100)
evaluate_purity_score(cluster_distances, label_cluster_map)
evaluate_gini_index(X_train, label_cluster_map)

converged at iter 32
evaluated objective is 26448.393157079583
evaluated purity score is 61.95166666666667%


0.9570461919444444

In [8]:
label_cluster_map, cluster_distances = k_means_clustering(X_train, y_train, k=20, max_iter=100)
evaluate_purity_score(cluster_distances, label_cluster_map)
evaluate_gini_index(X_train, label_cluster_map)

converged at iter 29
evaluated objective is 24911.90426408329
evaluated purity score is 63.92333333333333%


0.9734539794444445

## Section C - 20 NG

In [5]:
from sklearn.datasets import fetch_20newsgroups 
from sklearn.feature_extraction.text import TfidfVectorizer

newsgroups_dataset = fetch_20newsgroups(subset='train')
train_data = newsgroups_dataset.data
train_label = newsgroups_dataset.target

vectorizer = TfidfVectorizer()
train_data = vectorizer.fit_transform(train_data)
train_data = np.array(normalize(train_data.todense()))

In [ ]:
X_train = train_data[:5000]
y_train = train_label[:5000]

In [7]:
print(X_train.shape)
print(y_train.shape)

(60000, 784)
(60000,)


In [8]:
label_cluster_map, cluster_distances = k_means_clustering(X_train, y_train, k=5, max_iter=200)
evaluate_purity_score(cluster_distances, label_cluster_map)
evaluate_gini_index(X_train, label_cluster_map)

converged at iter 35
evaluated objective is 41960.91842235273
evaluated purity score is 37.79%


0.970444185

In [9]:
label_cluster_map, cluster_distances = k_means_clustering(X_train, y_train, k=10, max_iter=50)
evaluate_purity_score(cluster_distances, label_cluster_map)
evaluate_gini_index(X_train, label_cluster_map)

evaluated objective is 39450.29859381385
evaluated purity score is 55.655%


0.9657038086111112

In [10]:
label_cluster_map, cluster_distances = k_means_clustering(X_train, y_train, k=20, max_iter=50)
evaluate_purity_score(cluster_distances, label_cluster_map)
evaluate_gini_index(X_train, label_cluster_map)

evaluated objective is 37103.73234474686
evaluated purity score is 71.36333333333333%


0.9711141622222222